First I will Generate the mdp for row 1 solve row 2 solve and row 3 and row 4 solve and store it as an mdp

In [3]:
import random
import math
gmdp = [{},{},{}]
gpolicy =[{},{},{}]
gq_fn = [{},{},{}]
gval_fn = [{},{},{}]
gmove_reward = -1
gfinal_reward = 100
ggenerated = [set(),set(),set()]
gvisited_states = [set(),set(),set()]
terminal_states = set()
# Initialize the puzzle state
def init_state(rows_solved):
    val = [i for i in range(0, 16)]
    val[0] = '*'
    current_state = [[], [], [], []]
    for i in range(rows_solved):
        for j in range(1,5):
            current_state[i].append ((i)*4 +j)
            print((i)*4 +j)
            val.remove((i)*4 +j)
    for i in range(rows_solved,4):
        for j in range(4):
            x = val[random.randint(0, len(val) - 1)]
            current_state[i].append(x)
            val.remove(x)
    print("Initial State:")
    for row in current_state:
        print(row)
    return current_state

def gen_hash(state, row):
    hash = []
    if row <=1:
        req_list = [row*4 + 1, row*4 + 2, row*4 + 3, row*4 + 4, '*'] 
    else:
        req_list = [9,10,11,12,13,14,15,'*']
    for i in range(4):
        for j in range(4):
            x = state[i][j]
            if x not in req_list:
                hash.append(0)
            else:
                hash.append(x)
    return hash

def gen_random_policy(mdp, hashed, policy):
    tup_hash = tuple(hashed)
    policy[tup_hash] = random.choice(list(mdp[tup_hash].keys()))

def check_terminal(state,rows_solved):
    if(rows_solved<=1):
        for i in range(4):
            if state[rows_solved * 4 + i]!=4*rows_solved+i+1:
                return False
        return True
    else:
        for i in range(8,15):
            if state[i] != i+1:
                return False
        return True

def gen(mdp, hashed, move_reward, generated,rows_solved,policy):
    tup_hash = tuple(hashed)
    mdp[tup_hash] = {}
    pos = hashed.index('*')
    if(check_terminal(tup_hash,rows_solved)):
        mdp[tup_hash] = {i:(1,tup_hash,0,True) for i in range(4)}
        gen_random_policy(mdp, hashed, policy)
        generated.add(tup_hash)
        terminal_states.add(tup_hash)
        return 
    # Generate possible moves
    if pos % 4 != 0:  # Left move
        ex = hashed.copy()
        ex[pos] = hashed[pos - 1]
        ex[pos - 1] = '*'
        mdp[tup_hash][0] = (1, ex, move_reward if not check_terminal(ex,rows_solved) else gfinal_reward, check_terminal(ex,rows_solved))
    
    if pos % 4 != 3:  # Right move
        ex = hashed.copy()
        ex[pos] = hashed[pos + 1]
        ex[pos + 1] = '*'
        mdp[tup_hash][1] = (1, ex, move_reward if not check_terminal(ex,rows_solved) else gfinal_reward, check_terminal(ex,rows_solved))
    
    if pos // 4 != rows_solved:  # Up move
        ex = hashed.copy()
        ex[pos] = hashed[pos - 4]
        ex[pos - 4] = '*'
        mdp[tup_hash][2] = (1, ex, move_reward if not check_terminal(ex,rows_solved) else gfinal_reward, check_terminal(ex,rows_solved))
    
    if pos // 4 != 3:  # Down move
        ex = hashed.copy()
        ex[pos] = hashed[pos + 4]
        ex[pos + 4] = '*'
        mdp[tup_hash][3] = (1, ex,move_reward if not check_terminal(ex,rows_solved) else gfinal_reward, check_terminal(ex,rows_solved))
    
    gen_random_policy(mdp, hashed, policy)
    generated.add(tup_hash)

def mdp_gen(mdp, generated, rows_solved,  policy, gamma=0.9, delta=1e-6):
    current_state =[[],[],[],[]]
    current_state = init_state(rows_solved)
    hashed = gen_hash(current_state, rows_solved)
    tup_hash = tuple(hashed)
    gen(mdp, hashed, gmove_reward, generated,rows_solved,policy)
    for _ in range(55):
        for tup_hash, req in list(mdp.items()):
            for action, (prob, next_state, reward, terminal) in req.items():
                if tuple(next_state) not in generated:
                    gen(mdp, next_state, gmove_reward, generated,rows_solved,policy)
                    generated.add(tuple(next_state))
    if rows_solved==2 and len(terminal_states)==0:
        mdp = {}
        mdp_gen(mdp,generated,rows_solved,policy)

for i in range(3):
    mdp_gen(gmdp[i],ggenerated[i],i,gpolicy[i])
    with open(f"mdp{i}.txt", "w") as file:
        for item in gmdp[i]:
            file.write(f"{item} {gmdp[i][item]}\n") 
    print(i)

Initial State:
[8, 5, 3, 15]
[12, 1, 9, 7]
[10, 2, 4, 14]
[11, '*', 13, 6]
0
1
2
3
4
Initial State:
[1, 2, 3, 4]
[13, 10, 6, 8]
[12, 14, 9, '*']
[5, 7, 15, 11]
1
1
2
3
4
5
6
7
8
Initial State:
[1, 2, 3, 4]
[5, 6, 7, 8]
[13, 12, 11, 9]
[10, '*', 15, 14]
2


Now finding the optimal strategy by using Value Iteration

In [ ]:
import ast
import random
import math
import numpy as np


def value_iteration(mdp,policy, gamma=0.9, theta=1e-1):
    V = {key:0 for key in mdp}
    for j in range(60):
        print(j)
        x=0
        i=0
        Q  = {key:{action:0 for action in mdp[key]} for key in mdp}
        for key,dict in mdp.items():
            i+=1
            max_Q = -1000
            max_action = 0
            for action,(prob,next_state,reward,done) in dict.items():
                Q[key][action] +=prob *(reward + gamma*(not done)*V[tuple(next_state)])
                if Q[key][action]>=max_Q:
                    max_Q = Q[key][action]
                    max_action = action
                
                if abs(V[key] - max_Q)>theta:
                    x+=1
                V[key] = max_Q
                policy[key] = max_action
            
            if(i%100000==0):
                print(i/100000)
        if x==0:
            break              
    return policy,V
final_policy = [{},{},{}]
state_val = [{},{},{}]
for i in range(3):
    final_policy[i],state_val[i] = value_iteration(gmdp[i],gpolicy[i])

    with open(f"val{i}.txt", "w") as file:
        for item in state_val[i]:
            file.write(f"{item} {state_val[i][item]}\n")

    with open(f"policy{i}.txt", "w") as file:
        for item in final_policy[i]:
            file.write(f"{item} {final_policy[i][item]}\n")

0
1.0
2.0
3.0
4.0
5.0
1
1.0
2.0
3.0
4.0
5.0
2
1.0
2.0
3.0
4.0
5.0
3
1.0
2.0
3.0
4.0
5.0
4
1.0
2.0
3.0
4.0
5.0
5
1.0
2.0
3.0
4.0
5.0
6
1.0
2.0
3.0
4.0
5.0
7
1.0
2.0
3.0
4.0
5.0
8
1.0
2.0
3.0
4.0
5.0
9
1.0
2.0
3.0
4.0
5.0
10
1.0
2.0
3.0
4.0
5.0
11
1.0
2.0
3.0
4.0
5.0
12
1.0
2.0
3.0
4.0
5.0
13
1.0
2.0
3.0
4.0
5.0
14
1.0
2.0
3.0
4.0
5.0
15
1.0
2.0
3.0
4.0
5.0
16
1.0
2.0
3.0
4.0
5.0
17
1.0
2.0
3.0
4.0
5.0
18
1.0
2.0
3.0
4.0
5.0
19
1.0
2.0
3.0
4.0
5.0
20
1.0
2.0
3.0
4.0
5.0
21
1.0
2.0
3.0
4.0
5.0
22
1.0
2.0
3.0
4.0
5.0
23
1.0
2.0
3.0
4.0
5.0
24
1.0
2.0
3.0
4.0
5.0
25
1.0
2.0
3.0
4.0
5.0
26
1.0
2.0
3.0
4.0
5.0
27
1.0
2.0
3.0
4.0
5.0
28
1.0
2.0
3.0
4.0
5.0
29
1.0
2.0
3.0
4.0
5.0
30
1.0
2.0
3.0
4.0
5.0
31
1.0
2.0
3.0
4.0
5.0
32
1.0
2.0
3.0
4.0
5.0
33
1.0
2.0
3.0
4.0
5.0
34
1.0
2.0
3.0
4.0
5.0
35
1.0
2.0
3.0
4.0
5.0
36
1.0
2.0
3.0
4.0
5.0
37
1.0
2.0
3.0
4.0
5.0
38
1.0
2.0
3.0
4.0
5.0
39
1.0
2.0
3.0
4.0
5.0
40
1.0
2.0
3.0
4.0
5.0
41
1.0
2.0
3.0
4.0
5.0
42
1.0
2.0
3.0
4.0
5.0
43
1.0
2.0
3.0
4.0
5.

Now Playing the Game 

In [ ]:
import time
def left(current_state,rows_solved):
    new_state = [row.copy() for row in current_state]
    hashed = gen_hash(new_state, rows_solved)
    pos = hashed.index('*')
    if pos % 4 != 0:
        new_state[pos//4][pos%4], new_state[pos//4][pos%4-1] = new_state[pos//4][pos%4-1], new_state[pos//4][pos%4]
    return new_state

def right(current_state, rows_solved):
    new_state = [row.copy() for row in current_state]
    hashed = gen_hash(new_state, rows_solved)
    pos = hashed.index('*')
    if pos % 4 != 3:  # Ensure the move is valid
        new_state[pos // 4][pos % 4 + 1], new_state[pos // 4][pos % 4] = new_state[pos // 4][pos % 4], new_state[pos // 4][pos % 4 + 1]
    return new_state

def up(current_state, rows_solved):
    new_state = [row.copy() for row in current_state]
    hashed = gen_hash(new_state, rows_solved)
    pos = hashed.index('*')
    if pos // 4 != 0:  # Ensure the move is valid
        new_state[pos // 4 - 1][pos % 4], new_state[pos // 4][pos % 4] = new_state[pos // 4][pos % 4], new_state[pos // 4 - 1][pos % 4]
    return new_state

def down(current_state, rows_solved):
    new_state = [row.copy() for row in current_state]
    hashed = gen_hash(new_state, rows_solved)
    pos = hashed.index('*')
    if pos // 4 != 3:  # Ensure the move is valid
        new_state[pos // 4 + 1][pos % 4], new_state[pos // 4][pos % 4] = new_state[pos // 4][pos % 4], new_state[pos // 4 + 1][pos % 4]
    return new_state

def play_game(current_state, rows_solved,policy,mdp,move =0,reward =0):
    initial_state = [row.copy() for row in current_state]  
    hashed_tup = tuple(gen_hash(current_state, rows_solved))
    x=0
    while True:
        #time.sleep(0.3)
        print(f"Moves = {move} Reward = {reward}")
        print()
        if(x==1):
            break
        hashed_tup = tuple(gen_hash(current_state, rows_solved))
        if not hashed_tup in policy:
            print("The Puzzle seems to be unsolvable try some other puzzle")
            return current_state,move,reward,False
        action = policy[hashed_tup]
        if action == 0:
            current_state = left(current_state,rows_solved)
        elif action == 1:
            current_state = right(current_state,rows_solved)
        elif action == 2:
            current_state = up(current_state,rows_solved)
        elif action == 3:
            current_state = down(current_state,rows_solved)
        if mdp[hashed_tup][action][3]:  # Check if terminal state
            x+=1
        reward += mdp[hashed_tup][action][2]
        move+=1
        print_puzzle(current_state)
    return current_state,move,reward, True

def gen_hash(state, row):
    hash = []
    if row <=1:
        req_list = [row*4 + 1, row*4 + 2, row*4 + 3, row*4 + 4, '*'] 
    else:
        req_list = [9,10,11,12,13,14,15,'*']
    for i in range(4):
        for j in range(4):
            x = state[i][j]
            if x not in req_list:
                hash.append(0)
            else:
                hash.append(x)
    return hash

def print_puzzle(state):
    for row in state:
        print(" | ".join(str(cell if cell!='*' else " ").rjust(2) for cell in row))
    print()

val = [i for i in range(0, 16)]
val[0] = '*'
current_state = [[], [], [], []]
for i in range(4):
    for j in range(4):
        x = val[random.randint(0, len(val) - 1)]
        current_state[i].append(x)
        val.remove(x)
def check_solvable(current_state,rows_solved, policy,mdp):
    initial_state = [row.copy() for row in current_state]  
    x=0
    hashed_tup = tuple(gen_hash(current_state, i))
    while True:
        #time.sleep(0.4)
        if(x==1):
            break
        hashed_tup = tuple(gen_hash(current_state, rows_solved))
        if not hashed_tup in policy:
            return current_state, False
        action = policy[hashed_tup]
        if action == 0:
            current_state = left(current_state,rows_solved)
        elif action == 1:
            current_state = right(current_state,rows_solved)
        elif action == 2:
            current_state = up(current_state,rows_solved)
        elif action == 3:
            current_state = down(current_state,rows_solved)
        if mdp[hashed_tup][action][3]:  # Check if terminal state
            x+=1
    return current_state,True
check_state = current_state.copy()
solvable = True
print("The Puzzle:")
print_puzzle(current_state)
for i in range(3):
    check_state, solvable = check_solvable(check_state,i,gpolicy[i],gmdp[i])
    if not solvable:
        print("This puzzle is unsolvable!! Try some other puzzle")
move =0
reward =0
if(solvable):
    for i in range(3):
        current_state,move,reward,ok = play_game(current_state, i,final_policy[i],gmdp[i],move,reward) 
        if (i ==2 and ok):
            print("Hooray!! Solved the Puzzle")
        elif (i!=2):
            print(f"Done with row {i+1}")

NameError: name 'random' is not defined